# ImageNet Classification with Vision Transformer

Code from [here](https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/vit).

In [64]:
!pip install pillow

In [65]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests
import numpy as np
import tensorflow as tf
import imageio as iio

def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))


In [66]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


The model `vit-base-patch16-224` has 86.6M parameters https://huggingface.co/google/vit-base-patch16-224. <br>
The Vision Transformer (ViT) is a transformer encoder model (BERT-like)
* pretrained on a large collection of images in a supervised fashion: ImageNet-21k (14 million images, 21,843 classes) at resolution 224x224,
* fine-tuned on ImageNet 2012 (1 million images, 1,000 classes) at resolution 224x224.

ViT has 330 million parameters

### Load Images from **Local Computer Disk**

In [62]:
### ========= LOAD IMAGE from local laptop disk ================
from google.colab import files

uploaded = files.upload()   # starts a dialog to upload files

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

names = []            # uploaded image names
for key in uploaded:
  print(key)
  names.append(key)
image = iio.imread(names[0])

### Load Images from **Web**

You can obtain images from the [cocodataset](https://cocodataset.org/#explore) explorer and download the corresponding URLs. <br>
COCO 2017 train/val browser (123,287 images, 886,284 instances).

The next cell uploads from a local directory on your computer/laptop.

--2024-03-04 15:15:07--  http://farm1.staticflickr.com/49/158538590_b79a247333_z.jpg
Resolving farm1.staticflickr.com (farm1.staticflickr.com)... 108.158.0.70, 2600:9000:2753:9600:0:5a51:64c9:c681, 2600:9000:2753:6200:0:5a51:64c9:c681, ...
Connecting to farm1.staticflickr.com (farm1.staticflickr.com)|108.158.0.70|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm1.staticflickr.com/49/158538590_b79a247333_z.jpg [following]
--2024-03-04 15:15:07--  https://farm1.staticflickr.com/49/158538590_b79a247333_z.jpg
Connecting to farm1.staticflickr.com (farm1.staticflickr.com)|108.158.0.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘158538590_b79a247333_z.jpg.1’

158538590_b79a24733     [ <=>                ]  65.52K  --.-KB/s    in 0.002s  

2024-03-04 15:15:07 (38.6 MB/s) - ‘158538590_b79a247333_z.jpg.1’ saved [67090]



<ipython-input-77-cc02b75533b9>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = iio.imread("158538590_b79a247333_z.jpg")


array([[[248, 245, 252],
        [249, 246, 253],
        [249, 248, 254],
        ...,
        [150, 163, 180],
        [147, 160, 177],
        [147, 160, 177]],

       [[247, 244, 251],
        [247, 244, 251],
        [247, 246, 252],
        ...,
        [150, 163, 182],
        [147, 160, 179],
        [146, 159, 178]],

       [[244, 243, 249],
        [244, 243, 249],
        [245, 244, 250],
        ...,
        [148, 160, 182],
        [145, 157, 179],
        [145, 157, 179]],

       ...,

       [[ 50,  51,  56],
        [ 61,  62,  67],
        [ 59,  60,  65],
        ...,
        [ 18,  19,  11],
        [ 16,  17,   9],
        [ 16,  18,   7]],

       [[ 18,  17,  23],
        [ 26,  25,  31],
        [ 26,  26,  34],
        ...,
        [ 31,  31,  21],
        [ 30,  30,  20],
        [ 29,  29,  17]],

       [[ 27,  26,  32],
        [ 23,  22,  28],
        [ 21,  21,  29],
        ...,
        [ 41,  41,  31],
        [ 39,  39,  27],
        [ 37,  37,  25]]], dtype=uint8)
In [77]:
### ========= LOAD IMAGE from web ================
# e.g. address from  cocodataset explorer
!wget http://farm1.staticflickr.com/49/158538590_b79a247333_z.jpg
image = iio.imread("158538590_b79a247333_z.jpg")
image

### Load Image from **GitHub**

In [69]:
### ========= LOAD IMAGE from GitHub ================
!git clone https://github.com/gpaaxx/github_data.git
!ls ls github_data/05_images

fatal: destination path 'github_data' already exists and is not an empty directory.
ls: cannot access 'ls': No such file or directory
github_data/05_images:
airplane.jpg  boat.jpg	cat.jpg    ducks.jpg	   plate.jpg	  tennis_player.jpg
bakery.jpg    bus.jpg	couch.jpg  elephant.jpg    snowboard.jpg  umbrella.jpg
baseball.jpg  car.jpg	dog.jpg    park_bench.jpg  sparrow.jpg	  woman.jpg


<ipython-input-70-5633dcc3b767>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = iio.imread("github_data/05_images/airplane.jpg")


array([[[124, 151, 180],
        [124, 151, 180],
        [124, 151, 180],
        ...,
        [128, 159, 190],
        [128, 159, 190],
        [128, 159, 190]],

       [[124, 151, 180],
        [124, 151, 180],
        [124, 151, 180],
        ...,
        [128, 159, 190],
        [128, 159, 190],
        [128, 159, 190]],

       [[124, 151, 180],
        [124, 151, 180],
        [124, 151, 180],
        ...,
        [128, 159, 190],
        [128, 159, 190],
        [128, 159, 190]],

       ...,

       [[143, 159, 174],
        [143, 159, 174],
        [143, 159, 174],
        ...,
        [147, 165, 185],
        [147, 165, 185],
        [147, 165, 185]],

       [[143, 159, 174],
        [143, 159, 174],
        [143, 159, 174],
        ...,
        [147, 165, 185],
        [147, 165, 185],
        [147, 165, 185]],

       [[143, 159, 174],
        [143, 159, 174],
        [143, 159, 174],
        ...,
        [147, 165, 185],
        [147, 165, 185],
        [147, 165, 185]]], dtype=uint8)
In [70]:
image = iio.imread("github_data/05_images/airplane.jpg")
image

## Classify Image

In [78]:
#print("image.height,width   =",image.height, image.width)
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
#logits
predicted_class_idx = logits.argmax(-1).item()
lgt=logits.cpu().detach().numpy()[0]  # numpy array of logits
elgt=np.exp(lgt)
# find several classes
idx=(-lgt).argsort()[:5]
for id in idx:
    print("class id",id,"prob=",elgt[id]/np.sum(elgt),"\tname",model.config.id2label[id])

class id 442 prob= 0.76607776 	name bell cote, bell cot
class id 409 prob= 0.08831465 	name analog clock
class id 698 prob= 0.05831144 	name palace
class id 892 prob= 0.014862854 	name wall clock
class id 718 prob= 0.00587666 	name pier


In [7]:
feature_extractor??